# Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [7]:
#Import libraries
import numpy as np
import pandas as pd
import folium
import sklearn as skl
import requests
import geocoder
from geopy.geocoders import Nominatim

## Question 1

In [2]:
#Assign url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

#Retreive table from Wikipedia with read_html() method; this will be a list
wtable = pd.read_html(url, index_col=0,skiprows=1, attrs={"class":"wikitable"})  

#Table is contained in the first list element, then assigned to a variable for formatting
raw_table = wtable[0]

#Remove unassigned boroughs and rename columns 
test_table = raw_table[~raw_table[1].str.contains("Not assigned")].reset_index()
test_table.columns = ['PostalCode','Borough','Neighborhood']

#Iterate through rows and compare borough and neighbourhood values
for i in range(0,(test_table.shape[0])):
    if (test_table.iloc[i,:][1] != 'Not assigned') & (test_table.iloc[i,:][2] == 'Not assigned'):
        test_table.iloc[i, test_table.columns.get_loc('Neighborhood')] = test_table.iloc[i, test_table.columns.get_loc('Borough')]

#Group dataframe by postal code and borough to combine neighborhood values for one borough
df_toronto = test_table.groupby(['PostalCode', 'Borough'], as_index=False, sort=False).agg(', '.join) 


## * Please note: *   

### I could find no discernable sorting pattern for the final formatted dataframe, but all neighbourhood values were confirmed to have been consolidated. In other words, the *position* of the rows shown below differ from those shown in the Coursera example image. 

In [3]:
#Print first 15 rows of formatted dataframe
df_toronto.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [4]:
#Print formatted dataframe shape
print('Dataframe shape = {} rows, {} columns'.format(df_toronto.shape[0],df_toronto.shape[1]))

Dataframe shape = 103 rows, 3 columns


## Question 2

In [51]:
csv_coords = 'Geospatial_Coordinates.csv'
df_coords = pd.read_csv(csv_coords)
df_coords.columns = ['PostalCode','Latitude','Longitude']
df_coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [53]:
df_toronto.merge(df_coords, on='PostalCode', how='outer')

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [55]:
'''
#Failed attempt with geocoders.google() method, saved for posterity?

#initialize arrays to store coordinates
pclen = len(df_toronto.PostalCode)
ar_lat = np.zeros(pclen) 
ar_lon = np.zeros(pclen)

# initialize your variable to None
lat_lng_coords = [0,0]


for i in range(0,pclen+1):
    # loop until you get the coordinates
    while((lat_lng_coords[0] == 0) & (lat_lng_coords[1] == 0)):
        #g = geocoder.google('{}, Toronto, Ontario'.format(df_toronto.PostalCode[i]))
        g = geocoder.google('M3A, Toronto, Ontario')
        lat_lng_coords = g.latlng

        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    ar_lat[i] = latitude
    ar_lon[i] = longitude
'''
print('CHANGED')
    



CHANGED
